# Intro ag2, formerly Autogen
- [website](https://ag2.ai/)
- [github](https://github.com/ag2ai/ag2)
- [docs](https://docs.ag2.ai/docs/home/home)

In [ ]:
# !python -m pip install --upgrade ag2
# !python -m pip install --upgrade pydantic

In [2]:
# Import libraries
import os, autogen
from autogen import ConversableAgent #controller agent

print(f"autogen: {autogen.__version__}")

autogen: 0.2.25


In [3]:
# Set up LLM to be the control agent
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_config = {"model": "gpt-4o-mini"}

# Simple Q&A, single agent

In [4]:
# Generate a chat from an agent, without instantiation.
# This does not use the ConversableAgent class.
# There is no internal state!  So no memory of previous messages.

# Instantiate the ConversableAgent
agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)
# Generate a single session message.
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

# Generate another single session message.
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print()
print(reply)

Why did the scarecrow win an award?

Because he was outstanding in his field!

Sure, I can help with that! However, I need to know which joke you would like me to repeat. Could you please provide it?


# Simple, Multi-agents with Memory

In [5]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    # "stop words" to end conversation, other agent won't reply.
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    # "stop words" to end conversation, other agent won't reply.
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [8]:
# Instantiate w/Joe asking Cathy for a joke, and run 2 turns.
# Uses a controller agent, so memory of interactions is retained.
# Summarize information so far and pass forward to next turn.

# The agent flow interactions are defined by 1 initial message.
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=4, 
    # If you want to be able to inspect/summarize this.
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

# Automatically uses caching! So same input = same outputs.

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Alright, let’s keep that laughter flowing! What did the zero say to the eight? "Nice belt!" What’s your favorite type of joke?

--------------------------------------------------------------------------------
joe (to cathy):

Hey! I love that joke—classic! My favorite type of joke has to be puns. They’re like the dad bod of humor: a little cheesy but always brings a smile. Got any more jokes up your sleeve?

--------------------------------------------------------------------------------
cathy (to joe):

Absolutely, Joe! I’m all about that pun life! Here’s one for you: I used to be a baker, but I couldn't make enough dough. Now I'm just kneading some laughs instead! Got any favorites of your own?

--------------------------------------------------------------------------------
joe (to cathy):

That’s a good one! You r

In [10]:
# memory?
cathy.send(message="What's last joke we talked about?", 
recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------


# Parallel, Sequential Agents

Pattern requires an agent acting like PM, to carryover conversations.

E.g. 4 agents, including 1 manager.
- A = Designer
- B = Planner
- C = Formatter
- D = Manager

Pattern is:
A <> B, C <> B, D<>B, ->Summarizer, ->Chat result

In [11]:
# Set up LLM to be the control agent
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_config = {"model": "gpt-4o-mini"}

In [12]:
# Define 4 worker bots for a newsfeed, including manager.

# Get name, location, works in background
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

# Get news preferences, works in background
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

# Newsfeed generation agent, works in background
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

# Manager is also front facing, always asks for human input.
customer_proxy_agent = ConversableAgent(
    system_message='''You talk to customers. ''',
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [ ]:
# Create agent-to-agent flow using order of chats.
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            '''Hello, I'm here to help you get started with our product.
            Could you tell me your name and location?''',
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'topics': ''}",
        },
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [19]:
# Kick off the parallel, sequential agents
from autogen import initiate_chats

chat_results = initiate_chats(chats)
# chat_results = customer_proxy_agent.initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.
            Could you tell me your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

cy

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you, Cy! Could you please share your location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

dc

--------------------------------------------------------------------------------

************************************************

In [24]:
import pprint

print(f"Length conversation: {len(chat_results)}")
# View the chats
# pprint.pprint(chat_results.chat_history)
# Summarize, only if you turned it on during instantiation
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

# View token usage
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

Length conversation: 3
```json
{"name": "Cy", "location": "dc"}
```


```json
{
  "topics": "museums"
}
```


Cy from DC is interested in museums. Fun facts were shared about the Smithsonian Institution and the Louvre Museum, and a museum-related joke was provided. The conversation prompted engagement by asking about Cy's favorite museum or exhibit in DC.


{'usage_including_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 1146, 'completion_tokens': 186, 'total_tokens': 1332}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 822, 'completion_tokens': 134, 'total_tokens': 956}}}


{'usage_including_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 1272, 'completion_tokens': 186, 'total_tokens': 1458}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 1272, 'completion_tokens': 186, 't

In [ ]:
# # Try again with user input
# financial_tasks = [
#     """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?""",
#     """Investigate possible reasons of the stock performance leveraging market news.""",
# ]

# writing_tasks = ["""Develop an engaging blog post using any information provided."""]

# financial_assistant = autogen.AssistantAgent(
#     name="Financial_assistant",
#     llm_config=llm_config,
# )
# research_assistant = autogen.AssistantAgent(
#     name="Researcher",
#     llm_config=llm_config,
# )
# writer = autogen.AssistantAgent(
#     name="writer",
#     llm_config=llm_config,
#     system_message="""
#         You are a professional writer, known for
#         your insightful and engaging articles.
#         You transform complex concepts into compelling narratives.
#         Reply "TERMINATE" in the end when everything is done.
#         """,
# )

# user_proxy_auto = autogen.UserProxyAgent(
#     name="User_Proxy_Auto",
#     human_input_mode="NEVER",
#     is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
#     code_execution_config={
#         "last_n_messages": 1,
#         "work_dir": "tasks",
#         "use_docker": False,
#     },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
# )

# user_proxy = autogen.UserProxyAgent(
#     name="User_Proxy",
#     human_input_mode="ALWAYS",  # ask human for input at each step
#     is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
#     code_execution_config={
#         "last_n_messages": 1,
#         "work_dir": "tasks",
#         "use_docker": False,
#     },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
# )


# chat_results = autogen.initiate_chats([
#     {
#         "sender": user_proxy_auto,
#         "recipient": financial_assistant,
#         "message": financial_tasks[0],
#         "clear_history": True,
#         "silent": False,
#         "summary_method": "last_msg",
#     },
#     {
#         "sender": user_proxy_auto,
#         "recipient": research_assistant,
#         "message": financial_tasks[1],
#         "max_turns": 2,  # max number of turns for the conversation (added for demo purposes, generally not necessarily needed)
#         "summary_method": "reflection_with_llm",
#     },
#     {
#         "sender": user_proxy,
#         "recipient": writer,
#         "message": writing_tasks[0],
#         "carryover": "I want to include a figure or a table of data in the blogpost.",  # additional carryover to include to the conversation (added for demo purposes, generally not necessarily needed)
#     },
# ])

# Developer tools

In [20]:
import pprint

print(f"Length conversation: {len(chat_result.chat_history)}")
# View the chats
# pprint.pprint(chat_result.chat_history)
# Summarize, only if you turned it on during instantiation
pprint.pprint(chat_result.summary)

# View token usage
pprint.pprint(chat_result.cost)


Length conversation: 10
('The conversation highlights the enjoyment of puns and jokes, emphasizing how '
 'humor can bring smiles and lightness to life. Both participants share their '
 'favorite jokes and appreciate the delivery and cleverness of wordplay, '
 'fostering a fun and playful exchange.')
{'usage_excluding_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 1799,
                                                                 'cost': 0,
                                                                 'prompt_tokens': 27138,
                                                                 'total_tokens': 28937},
                                      'total_cost': 0},
 'usage_including_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 2170,
                                                                 'cost': 0,
                                                                 'prompt_tokens': 28526,
                                       